In [1]:
import numpy as np
import pandas as pd
import polars as pl
import plotly.express as px
import plotly.graph_objects as go
from great_tables import GT, md

In [2]:
def direct_proportion_normalize(series, direction='positive'):
            """
            Normaliza una serie de datos usando el método de Proporción Directa (Normalización a la Suma).
            Implementa un 'shift' para asegurar que todos los valores sean no negativos.
            
            - Si la dirección es 'positive' (Alto=Bueno), usa Proporción Directa.
            - Si la dirección es 'negative' (Alto=Malo), usa una Proporción Inversa Suavizada 
            (penalizando valores altos) basada en la fórmula R/(R+X) donde R es la media + std.
            """
            
            # Asegurar que el input es una Serie de Pandas
            if not isinstance(series, pd.Series):
                series = pd.Series(series)

            # 1. SHIFTING: Asegurar que el valor mínimo de la serie sea 0 o positivo.
            R = series.mean() + series.std()*3

            min_val = series.min()
            
            # Shift para que el mínimo sea exactamente 0 (si era negativo)
            if min_val < 0:
                # Corrimiento de prom + stdev.
                shifted_series = series + R 
            else:
                # No se necesita shift si el mínimo es 0 o positivo.
                shifted_series = series
            
            # --- Lógica de Normalización ---
            
            if direction == 'positive':
                # Alto=Bueno (Proporción Directa Estándar)
                total_sum = shifted_series.sum()
                
                if total_sum == 0:
                    # Caso extremo: si todos los valores son cero después del shift.
                    return pd.Series(1.0 / len(shifted_series), index=shifted_series.index)
                
                return shifted_series / total_sum
            
            elif direction == 'negative':
                
                # Alto=Malo: Proporción Inversa Suavizada (penalizando valores altos)
                # Un valor grande (Alto=Malo) dará un resultado penalizado (cercano a 0).
                penalized_series = 1 / shifted_series
                
                # 3. Normalizar la serie penalizada a la suma (Proporción Directa)
                total_sum_penalized = penalized_series.sum()
                
                if total_sum_penalized == 0:
                    # Si, por alguna razón, la suma es 0 después de la penalización (muy improbable con R>0)
                    return pd.Series(1.0 / len(shifted_series), index=shifted_series.index)
                    
                return penalized_series / total_sum_penalized

In [3]:
def calculate_index(df, weights, presupuesto):
            """
            Calcula la Asignación de Fondo Ponderada (Reparto Directo) y la contribución monetaria por variable.
            """
            
            # 1. Fondo restante a repartir por variables (94% del fondo)
            #w_remaining_fund_percent = 1 - weights['Monto base']
            #w_remaining_fund = presupuesto * w_remaining_fund_percent
            
            # Diccionario para almacenar las contribuciones monetarias de cada variable
            contributions = {}

            # --- Definiciones de Normalización y Cálculo de Monto (94% del fondo) ---    
            # Mapping of variable names to their properties
            variable_map = {
                'Pob': 'positive', 'Tasa_policial': 'positive', 'Profesionalizacion': 'positive', 
                'Ctrl_conf': 'positive', 'Disp_camaras': 'positive', 'Disp_lectores_veh': 'positive',
                'Cump_presup': 'positive', 'Servs_forenses': 'positive', 'Eficiencia_procesal': 'positive',
                'Inc_del': 'positive', 'Dig_salarial': 'positive', 
                'Tasa_abandono_llamadas': 'negative', 'Sobrepob_penitenciaria': 'positive', 
                'Proc_justicia': 'negative'
            }

            for var_name, direction in variable_map.items():
                # 1. Normalización
                df[f'{var_name}_prop'] = direct_proportion_normalize(df[var_name], direction=direction)
                
                # 2. Cálculo de la Contribución Monetaria Ponderada
                # (Proporción * Peso de la variable * Fondo restante)
                contribution_col_name = f'Monto_{var_name}'
                contributions[contribution_col_name] = df[f'{var_name}_prop'] * weights[var_name] * presupuesto

            # --- Cálculo del Monto Base (6% del fondo) ---
            w_base_amount = presupuesto * weights['Monto base']
            base_share = w_base_amount / len(df)
            
            # sumamos el monto base constante para cada fila
            contributions['Monto_Base'] = pd.Series(base_share, index=df.index) 

            # --- 3. Cálculo de la Asignación Bruta y Reparto Final ---
            # Sumar todas las contribuciones. (Esto incluye las 14 Series de variables + la Serie de Monto Base)
            # Al ser todas Series, la suma es vectorial (fila por fila).
            df['Asignacion_Bruta'] = sum(contributions.values())
            
            # Combinar todas las contribuciones (Monto_X) con el DF principal
            df = pd.concat([df, pd.DataFrame(contributions)], axis=1)
            
            # Para el resto del código (remanente) se necesita una columna que sume 1.00 y represente el reparto:
            df['Reparto'] = df['Asignacion_Bruta'] / df['Asignacion_Bruta'].sum()
            
            return df

In [5]:
fasp_datos_entrada = pd.read_csv('fasp_datos_entrada.csv')

In [8]:
presupuesto = 9_941_162_915.0

In [6]:
weights = {
    'Pob': 0.075,
    'Inc_del': 0.21,
    'Tasa_policial': 0.045,
    'Dig_salarial': 0.045,
    'Profesionalizacion': 0.135,
    'Ctrl_conf': 0.0225,
    'Disp_camaras': 0.0788,
    'Disp_lectores_veh': 0.0788,
    'Tasa_abandono_llamadas': 0.045,
    'Cump_presup': 0.005,
    'Sobrepob_penitenciaria': 0.0368,
    'Proc_justicia': 0.0858,
    'Servs_forenses': 0.0368,
    'Eficiencia_procesal': 0.0858,
    'Monto base': 0.015,
}

In [9]:
df_results = calculate_index(fasp_datos_entrada, weights, presupuesto)

In [10]:
df_results

,Entidad_Federativa,Pob,Inc_del,Tasa_policial,Dig_salarial,Profesionalizacion,Ctrl_conf,Disp_camaras,Disp_lectores_veh,Tasa_abandono_llamadas,...,Monto_Cump_presup,Monto_Servs_forenses,Monto_Eficiencia_procesal,Monto_Inc_del,Monto_Dig_salarial,Monto_Tasa_abandono_llamadas,Monto_Sobrepob_penitenciaria,Monto_Proc_justicia,Monto_Base,Reparto
0,Aguascalientes,1567559,0.446490,1.013040,-0.100692,70,0.817436,0.95,0.95,0.005837,...,1.409119e+06,1.180287e+07,4.121167e+07,1.304116e+08,1.402225e+07,5.138602e+07,1.148098e+07,1.242907e+07,4.659920e+06,0.039694
1,Baja California,4189285,0.458217,0.310554,0.077375,90,0.840051,0.95,0.85,0.017397,...,1.379316e+06,9.717327e+06,1.383062e+07,1.338366e+08,1.879980e+07,1.724105e+07,9.698634e+06,1.183031e+07,4.659920e+06,0.035169
2,Baja California Sur,921734,0.240091,0.557645,-0.328636,50,0.699760,0.80,0.90,0.042804,...,1.177507e+06,1.122257e+07,4.222259e+07,7.012613e+07,7.906476e+06,7.007185e+06,8.410501e+06,1.290423e+07,4.659920e+06,0.025858
3,Campeche,962014,0.070165,1.280647,-0.364872,50,0.935152,0.85,0.90,0.013009,...,1.428899e+06,1.257054e+07,3.965417e+07,2.049399e+07,6.934259e+06,2.305608e+07,8.052748e+06,1.882605e+07,4.659920e+06,0.024003
4,Coahuila,3440385,0.068016,0.844092,-0.178596,120,0.908472,0.65,0.90,0.041578,...,1.382331e+06,1.235152e+07,1.783855e+07,1.986611e+07,1.193207e+07,7.213858e+06,1.258391e+07,9.302595e+06,4.659920e+06,0.023578
5,Colima,771538,0.400888,0.799701,-0.244043,50,0.901290,0.95,0.96,0.014687,...,1.638462e+06,1.182701e+07,2.735486e+07,1.170919e+08,1.017613e+07,2.042253e+07,6.954824e+06,2.139083e+07,4.659920e+06,0.032458
6,Chiapas,6185469,0.030636,1.588238,-0.222880,50,0.818182,0.80,0.75,0.092701,...,2.647726e+06,1.055257e+07,1.956715e+07,8.948303e+06,1.074393e+07,3.235536e+06,1.319785e+07,2.140379e+07,4.659920e+06,0.022429
7,Chihuahua,4087306,0.215912,0.800038,-0.131304,150,0.868823,0.98,0.80,0.015639,...,1.849058e+06,1.138886e+07,2.716716e+07,6.306402e+07,1.320093e+07,1.917819e+07,1.098624e+07,1.818140e+07,4.659920e+06,0.033136
8,Ciudad de México,9159966,0.336235,8.597412,-0.143149,150,0.792322,0.95,0.90,0.030098,...,1.357900e+06,1.082623e+07,4.129025e+07,9.820800e+07,1.288312e+07,9.965418e+06,9.922585e+06,3.808617e+07,4.659920e+06,0.050455
9,Durango,1939482,0.076257,0.685750,-0.152691,70,0.814666,0.75,0.90,0.057034,...,1.358820e+06,1.216962e+07,2.482909e+07,2.227340e+07,1.262711e+07,5.258895e+06,1.632681e+07,1.880028e+07,4.659920e+06,0.022658
